In [1]:
import tkinter as tk
from tkinter import ttk
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from matplotlib.collections import PatchCollection
import time
from enum import Enum
import numpy as np

In [2]:
class State(Enum):
    SOURCE = 0
    SUSCEPTIBLE = 1
    INFECTED = 2
    RECOVERED = 3
    
STATE2COLOR = {
State.SOURCE: "red",
State.SUSCEPTIBLE: "grey",
State.INFECTED: "orange",
State.RECOVERED: "green"
}

In [3]:
sg = None  # Тут буде зберігатися граф
sg_copy = None  # Копія графа
fig = None

In [4]:
def validate_inputs():
    """Перевіряє коректність введених даних."""
    try:
        count = int(entry.get())
        if count < 1:
            label_status.config(text="Кількість осіб має бути цілим числом більше 0.")
            return False
    except ValueError:
        label_status.config(text="Кількість осіб має бути цілим числом.")
        return False

    try:
        probability = float(entry_2.get())
        if not (0 <= probability <= 1):
            label_status.config(text="Ймовірність має бути числом у межах від 0 до 1.")
            return False
    except ValueError:
        label_status.config(text="Ймовірність має бути числом у межах від 0 до 1.")
        return False

    if combo_box.get() not in ["random graph", "scale free graph"]:
        label_status.config(text="Оберіть коректний тип графа.")
        return False

    label_status.config(text="")  # Очищення повідомлення про помилки
    return True

In [5]:
def draw_plot():
    global sg, node_patches, fig
    """Створює інтерактивний граф NetworkX і відображає його у Canvas."""
    if not validate_inputs():
        return

    try:
        # Отримання значень з полів вводу
        count = int(entry.get())
        probability = float(entry_2.get())
        graph_type = combo_box.get()

        # Очищення Canvas
        for widget in canvas_frame.winfo_children():
            widget.destroy()

        # Створення графа на основі обраного типу
        if graph_type == "random graph":
            sg = nx.fast_gnp_random_graph(n=count, p=probability, seed=1, directed=True)
        elif graph_type == "scale free graph":
            sg = nx.scale_free_graph(n=count, seed=1).to_directed()

        pos = nx.spring_layout(sg, seed=5)

        # Ініціалізація властивостей вузлів
        np.random.seed(1)
        for node in sg.nodes:
            sg.nodes[node]["influence"] = len(list(sg.neighbors(node)))
            sg.nodes[node]["state"] = State.SUSCEPTIBLE
            sg.nodes[node]["resistance"] = np.random.random()

        # Побудова графа
        fig, ax = plt.subplots(figsize=(5, 5))
        ax.axis("off")

        # Малювання вузлів
        node_patches = []
        for node, (x, y) in pos.items():
            circle = Circle((x, y), 0.05, color=STATE2COLOR[sg.nodes[node]["state"]], picker=True)
            ax.add_patch(circle)
            node_patches.append((node, circle))

        # Малювання ребер зі стрілками
        for u, v in sg.edges:
            x1, y1 = pos[u]
            x2, y2 = pos[v]
            arrowprops = dict(arrowstyle="->", color="black", lw=1, shrinkA=5, shrinkB=5)
            ax.annotate("", xy=(x2, y2), xytext=(x1, y1), arrowprops=arrowprops)

        # Налаштування автоматичних меж
        ax.set_aspect("equal")
        ax.autoscale()

        def on_pick(event):
#     if is_simulating:  # Якщо йде симуляція, блокуємо зміну стану
#         return
    
            artist = event.artist
            for node, circle in node_patches:
                if circle == artist:
                    current_state = sg.nodes[node]["state"]

        #                     next_state = State((current_state.value + 1) % len(State))

                    # Перемикаємо лише між червоним і сірим
                    if current_state == State.SUSCEPTIBLE:
                        next_state = State.SOURCE
                    else:
                        next_state = State.SUSCEPTIBLE

                    sg.nodes[node]["state"] = next_state
                    circle.set_facecolor(STATE2COLOR[next_state])
                    fig.canvas.draw()
                    break
        
#         Підключення обробника подій
        fig.canvas.mpl_connect("pick_event", on_pick)
#         pick_event_handler = fig.canvas.mpl_connect("pick_event", on_pick)



        # Відображення графіка у Canvas
        canvas_widget = FigureCanvasTkAgg(fig, master=canvas_frame)
        canvas_widget.draw()
        canvas_widget.get_tk_widget().pack(fill=tk.BOTH, expand=True)
        label_status.config(text="Граф успішно згенеровано!\nОберіть людей, які поширюватимуть дезінформацію", fg="green")

    except Exception as e:
        label_status.config(text=f"Помилка при створенні графа: {str(e)}", fg="red")


In [6]:
# Функція для зміни стану вузла


In [7]:
def zoom(event):
    """Масштабує Canvas, якщо графік вже завантажений."""
    scale = 1.0
    if event.delta > 0:  # Збільшення
        scale = 1.1
    elif event.delta < 0:  # Зменшення
        scale = 0.9
    canvas.scale("all", 0, 0, scale, scale)

In [8]:
def select(event):
    selected_item = combo_box.get()

In [9]:
def start_simulation():
    global sg, sg_copy, fig
    
    steps = int(entry_4.get())
    
    sg_copy = sg.copy()
    
    for step in range(steps):
        # Оновлюємо стани вузлів
        for node in sg.nodes:
            update_state(sg, sg_copy, node)

        # Оновлюємо стани в оригінальному графі
        for node in sg.nodes:
            sg.nodes[node]["state"] = sg_copy.nodes[node]["state"]
        
        # Оновлюємо кольори вузлів
        update_node_colors()

        # Візуалізуємо новий стан графа
        fig.canvas.draw()
        plt.pause(2)  # Затримка для візуалізації

        
    label_status.config(text="Симуляція завершена", fg="green")

In [10]:
# Функція для оновлення кольорів вузлів
def update_node_colors():
    global sg, node_patches
    
    for node, circle in node_patches:
        state = sg.nodes[node]["state"]
        color = STATE2COLOR[state]  # Отримуємо колір відповідно до стану
        circle.set_facecolor(color)


In [11]:
# Функція для зміни стану вузла
def update_state(sg, sg_copy, node):
    successors = set(sg.neighbors(node))
    predecessors = set(nx.all_neighbors(sg, node)) - successors
    state = sg.nodes[node]["state"]
    
    if state == State.SOURCE:
        return
        
    elif state == State.RECOVERED:
        condition = np.random.random()
        if sg.nodes[node]["resistance"] > condition:
            sg.nodes[node]["resistance"] = min(sg.nodes[node]["resistance"] * 2, sg.nodes[node]["resistance"] + np.random.random(), 1)
        else:
            sg_copy.nodes[node]["state"] = State.SUSCEPTIBLE
            
    elif state == State.SUSCEPTIBLE:
        source_influenced = State.SOURCE in [sg_copy.nodes[pre]["state"] for pre in predecessors]
        infected_influenced = State.INFECTED in [sg_copy.nodes[pre]["state"] for pre in predecessors]
        if source_influenced or infected_influenced:
            condition = np.random.random()
            if sg.nodes[node]["resistance"] < condition:
                sg_copy.nodes[node]["state"] = State.INFECTED
                
    elif state == State.INFECTED:
        condition = np.random.random()
        if sg.nodes[node]["resistance"] > condition:
            sg_copy.nodes[node]["state"] = State.RECOVERED
        else:
            sg.nodes[node]["resistance"] = max(sg.nodes[node]["resistance"]/2, sg.nodes[node]["resistance"] - np.random.random())
            
    else:
        print("Unsupported state, exit.")

In [ ]:
# is_simulating = False


# Головне вікно
root = tk.Tk()
root.title("Prototype")
root.geometry("900x600")

title_label = tk.Label(root, text="Графова модель", font=("Arial", 12, "bold"))
title_label.pack(pady=10)

# Ліва частина з Canvas
left_frame = tk.Frame(root)
left_frame.pack(side="left", fill="both", expand=True)

canvas_frame = tk.Frame(left_frame, bg="white")
canvas_frame.pack(fill="both", expand=True)

# Права частина з полем вводу і кнопкою
right_frame = tk.Frame(root)
right_frame.pack(side="right", fill="both", padx=20, pady=15)

# Заголовок
title_label = tk.Label(right_frame, text="Створення мережі", font=("Arial", 10))
title_label.pack(pady=10)



entry_frame = tk.Frame(right_frame)
entry_frame.pack(pady=5)

label = tk.Label(entry_frame, text="Кількість осіб:")
label.pack(side="left", padx=5)

entry = tk.Entry(entry_frame, width=10)
entry.pack(side="left")



entry_frame_2 = tk.Frame(right_frame)
entry_frame_2.pack(pady=5)

label_2 = tk.Label(entry_frame_2, text="Ймовірність з'єднання:")
label_2.pack(side="left", padx=5)

entry_2 = tk.Entry(entry_frame_2, width=10)
entry_2.pack(side="left")



entry_frame_3 = tk.Frame(right_frame)
entry_frame_3.pack(pady=5)

label_3 = tk.Label(entry_frame_3, text="Тип графу: ")
label_3.pack(side="left", padx=5)

combo_box = ttk.Combobox(entry_frame_3, values=["random graph", "scale free graph"])
combo_box.pack(pady=5)

combo_box.set("random graph")


combo_box.bind("<<ComboboxSelected>>", select)



button_draw = tk.Button(right_frame, text="Згенерувати граф", command=draw_plot)
button_draw.pack(pady=5)



entry_frame_4 = tk.Frame(right_frame)
entry_frame_4.pack(pady=5)

label_4 = tk.Label(entry_frame_4, text="Кількість кроків симуляції:")
label_4.pack(side="left", padx=5, pady=20)

entry_4 = tk.Entry(entry_frame_4, width=10)
entry_4.pack(side="left")



button_simulate = tk.Button(right_frame, text="Запустити симуляцію", command=start_simulation)
button_simulate.pack(pady=5)



label_status = tk.Label(right_frame, text="", fg="red")
label_status.pack(pady=5)


# Запуск програми
root.mainloop()